In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# read part 1
df = pd.read_pickle('../M5_Uncertainty_3rd_place/processed_data/grid_part_1_eval.pkl')
df = df[df['dept_id'] == 'HOBBIES_2']
# 将categories转换为数字
df['id'] = df['id'].cat.codes
df['item_id'] = df['item_id'].cat.codes
df['dept_id'] = df['dept_id'].cat.codes
df['cat_id'] = df['cat_id'].cat.codes
df['store_id'] = df['store_id'].cat.codes
df['state_id'] = df['state_id'].cat.codes
df
data = df.values
data = data[:11000, :]
data_inp = data
data_inp[:,7] = -1
input_num = len(data[0])

test = data[10000:11000, :]

del df

In [3]:
# emmbedding
torch.manual_seed(0)
np.random.seed(0)

In [4]:
# build an attention 
class M5Net(torch.nn.Module):
    def __init__(self):
        super(M5Net, self).__init__()
        self.emmbedding = torch.nn.Embedding(input_num, 10)
        self.q = torch.nn.Linear(9, 10)
        self.k = torch.nn.Linear(9, 10)
        self.v = torch.nn.Linear(9, 10)
        self.fc = torch.nn.Linear(10, 1)

    def forward(self, x):
        # x = self.emmbedding(x)
        # x = x.view(x.size(0), -1, 10)
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        x = torch.matmul(q, k.T)
        x = torch.nn.functional.softmax(x, dim=-1)
        x = torch.matmul(x, v)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# train
data = torch.tensor(data_inp, dtype=torch.float).to(device)
model = M5Net().to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()

for epoch in range(1000):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, torch.tensor(data, dtype=torch.float).to(device))
    loss.backward()
    optimizer.step()
    print(loss.item())
print(output)

/tmp/ipykernel_30072/3929953920.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = criterion(output, torch.tensor(data, dtype=torch.float).to(device))
/home/gabe/DEEPLEARNING/deep/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([11000, 9])) that is different to the input size (torch.Size([11000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


62885348.0
55459364.0
49404684.0
44628728.0
41026676.0
38491720.0
36905248.0
36129264.0
36001876.0
36336100.0
36927628.0
37577816.0
38127972.0
38481368.0
38604380.0
38512800.0
38252984.0
37884580.0
37467712.0
37054660.0
36685600.0
36386844.0
36171384.0
36040468.0
35986060.0
35993628.0
36045104.0
36121384.0
36204584.0
36279892.0
36336512.0
36368152.0
36372780.0
36352372.0
36311516.0
36256776.0
36195380.0
36134324.0
36079576.0
36035488.0
36004532.0
35987288.0
35982568.0
35987932.0
36000096.0
36015412.0
36030528.0
36042692.0
36050116.0
36051980.0
36048484.0
36040560.0
36029736.0
36017676.0
36005988.0
35996004.0
35988624.0
35984164.0
35982564.0
35983308.0
35985708.0
35988896.0
35992080.0
35994620.0
35996140.0
35996428.0
35995524.0
35993696.0
35991312.0
35988760.0
35986372.0
35984444.0
35983208.0
35982600.0
35982608.0
35983060.0
35983768.0
35984492.0
35985120.0
35985480.0
35985544.0
35985328.0
35984900.0
35984348.0
35983768.0
35983224.0
35982816.0
35982596.0
35982520.0
35982588.0
35982740.0

In [5]:
test[:, 7] = -1
test = torch.tensor(test, dtype=torch.float).to(device)
model.eval()
output = model(test)
print(output)

tensor([[2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358.9773],
        [2358